<a href="https://colab.research.google.com/github/brighamfrandsen/econ484/blob/master/examples/06 Lasso (movies).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Import necessary libraries
!git clone https://github.com/brighamfrandsen/econ484.git
%cd econ484/utilities
from preamble import *
%cd ../data

Cloning into 'econ484'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 321 (delta 63), reused 67 (delta 22), pack-reused 140 (from 1)
Receiving objects: 100% (321/321), 6.93 MiB | 10.25 MiB/s, done.
Resolving deltas: 100% (119/119), done.
/content/econ484/data/econ484/utilities
/content/econ484/data/econ484/data


# Example: Movie-going and Weather

This notebook will illustrate the entire supervised machine learning process in the context of predicting movie attendance based on the weather on opening weekend.

## 1. Figure out your question

How many people would be expected to attend a movie on a weekend with temperatures in the X1s, precipitation of X2, humidity of X3, . . . ?

## 2. Obtain a labeled dataset

In [17]:
import pandas as pd
import numpy as np

In [18]:
moviedata=pd.read_csv('./opening_wkend.csv')

print(moviedata.head())
print("Shape: {}".format(str(moviedata.shape)))

   tickets_wk1d_r  tickets_ratedgpg_wk1d_r  tickets_adult_wk1d_r  \
0           -1.72                    -0.71                 -1.60   
1           -1.31                      NaN                 -0.82   
2            0.96                      NaN                  0.98   
3           -1.31                      NaN                 -1.07   
4            2.09                    -0.01                  1.65   

   tickets_p33_highbudget_wk1d_r  ...  res_own_mat10_80_0  res_own_mat10_90_5  \
0                            NaN  ...               -0.02            2.67e-03   
1                            NaN  ...               -0.06           -4.79e-03   
2                           0.55  ...                0.03            3.01e-02   
3                          -0.97  ...                0.09           -6.76e-02   
4                           1.73  ...               -0.17            9.77e-02   

   res_own_mat10_90_6  res_own_mat10_90_0  
0            2.46e-03           -2.91e-04  
1           -3.9

Let's define our "label" (y) vector and our "feature" matrix (X):

In [19]:
y = moviedata.filter(items=['tickets_wk1d_r'])
X = moviedata.filter(like='res_own',axis=1)
print('our y vector is:\n',y.head)
print('our X matrix is:\n',X.head)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

our y vector is:
 <bound method NDFrame.head of       tickets_wk1d_r
0              -1.72
1              -1.31
2               0.96
3              -1.31
4               2.09
...              ...
1666            2.58
1667            6.18
1668           -2.35
1669           -2.06
1670           -1.98

[1671 rows x 1 columns]>
our X matrix is:
 <bound method NDFrame.head of       res_own_snow  res_own_rain  res_own_mat5_10  res_own_mat5_15  ...  \
0        -1.16e-01         -0.08        -2.46e-02        -2.72e-02  ...   
1         4.16e-02         -0.10         2.21e-03         1.53e-03  ...   
2         6.56e-02          0.14         1.57e-03         1.65e-03  ...   
3         2.26e-04         -0.10         1.54e-03         1.65e-03  ...   
4        -1.54e-04         -0.04         1.58e-03         1.67e-03  ...   
...            ...           ...              ...              ...  ...   
1666     -3.11e-02         -0.06         4.45e-03         2.09e-03  ...   
1667     -3.38e-02        

## 3. Divide into training and test sets

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,test_size=.25)
y_train.shape

(1253, 1)

## 4. Pick an appropriate method

In [21]:
from sklearn.linear_model import Lasso

## 5. Choose regularization parameters via cross-validation on the training set

By hand if you really want:

In [ ]:
from sklearn.model_selection import cross_val_score

In [25]:
lasso = Lasso(alpha=0.032, max_iter=100)
scores = cross_val_score(lasso,X_train,y_train,cv=5)
print("Cross-validation scores: {}".format(scores))
print("Average cross-validation score: {:.4f}".format(scores.mean()))

Cross-validation scores: [ 0.044 -0.018  0.043  0.011  0.05 ]
Average cross-validation score: 0.0260


Or use GridSearchCV and do it automatically:

In [26]:
from sklearn.model_selection import GridSearchCV
# define grid for alpha
param_grid = {'alpha': [.0001, .005,.01, .015,.02, .025, .03, .031, .032, .034, .035, .036, .04 ,.05, .06 ],'max_iter': [1000]}
grid_search = GridSearchCV(Lasso(),param_grid,cv=5,return_train_score=True)
grid_search.fit(X_train,y_train)
print("Best alpha: ",grid_search.best_estimator_.get_params()['alpha'])

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.718e+02, tolerance: 1.358e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.859e+02, tolerance: 1.374e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.794e+02, tolerance: 1.365e

Best alpha:  0.032


Or, even easier, just use LassoCV:

In [27]:
from sklearn.linear_model import LassoCV
lassocv = LassoCV(cv=5,max_iter=1000).fit(X_train, y_train)
print(lassocv.alpha_)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:1664: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.21119559708586166, tolerance: 0.13580844599209113
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.17314731759279312, tolerance: 0.13580844599209113
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Object

0.03162543390583107


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 100.99310621831216, tolerance: 0.14134953469485165
  model = cd_fast.enet_coordinate_descent_gram(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 114.2567875136217, tolerance: 0.14134953469485165
  model = cd_fast.enet_coordinate_descent_gram(


## 6. Fit model on whole training set using the cross-validated parameters

In [28]:
lassowcvalpha=Lasso(alpha = lassocv.alpha_,max_iter=100000).fit(X_train,y_train)

In [29]:
lassowcvalpha.coef_

array([-0.   ,  0.   , -0.   ,  0.   ,  0.   ,  0.   , -0.   , -0.   ,
       -0.   , -0.   ,  0.   ,  0.   , -0.   ,  0.   ,  0.   , -0.   ,
        0.   ,  0.   ,  0.   , -0.   , -0.   , -0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   , -0.024, -0.   , -0.   ,  0.   , -0.002,
        0.   ,  0.   , -0.   , -0.   , -0.   ,  0.   ,  0.   , -0.   ,
       -0.   ,  0.   , -0.   ,  0.   ,  0.   , -0.   , -0.   , -0.   ,
        0.   , -0.   ,  0.   , -0.   ,  0.   ,  0.   , -0.052,  0.01 ,
        0.001,  0.   , -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
       -0.007, -0.   , -0.   ,  0.   , -0.   , -0.   , -0.   , -0.   ,
       -0.   , -0.   , -0.009, -0.   , -0.   ,  0.102,  0.   , -0.   ,
        0.   ,  0.027, -0.   ,  0.   , -0.   ,  0.   ,  0.   , -0.   ,
        0.   , -0.   ,  0.   , -0.012, -0.033, -0.081,  0.   , -0.012,
        0.   ,  0.03 ,  0.   , -0.   , -0.015,  0.   ,  0.05 , -0.   ,
       -0.   , -0.   ,  0.   ,  0.   ,  0.   , -0.   , -0.   , -0.036,
      

## 7. Evaluate model by applying it to test set

In [30]:
print('Lasso score on test set: {:.4f}'.format(lassowcvalpha.score(X_test,y_test)))

Lasso score on test set: 0.0543


## 8. Repeat 4-7 for several methods

In [32]:
from sklearn.linear_model import RidgeCV
ridgecv = RidgeCV().fit(X_train,y_train)
ridgecv.score(X_test,y_test)

0.049370515025077055

In [33]:
ridgecv.coef_

array([-0.028, -0.003,  0.004, -0.04 ,  0.037, -0.023, -0.003, -0.017,
       -0.006,  0.02 ,  0.027, -0.034,  0.024, -0.018,  0.029, -0.018,
        0.016, -0.01 , -0.002, -0.019, -0.006,  0.023, -0.023,  0.031,
        0.024, -0.022,  0.042, -0.066, -0.001,  0.003,  0.011, -0.031,
       -0.004, -0.014, -0.   ,  0.058, -0.042, -0.016, -0.   , -0.012,
        0.01 , -0.002,  0.004,  0.008,  0.004, -0.012,  0.059, -0.028,
        0.173, -0.02 ,  0.03 ,  0.169, -0.116,  1.232,  0.501,  0.093,
        0.607,  0.415, -0.035,  0.687,  0.561, -0.063,  0.803,  0.49 ,
       -0.127,  0.679,  0.293,  0.022,  0.392,  0.315, -0.094,  0.424,
        0.236,  0.045, -0.056, -0.031,  0.18 ,  0.154, -0.17 ,  0.043,
       -0.229, -0.053, -0.037, -0.183, -0.215, -0.046, -0.345, -0.163,
       -0.078, -0.44 , -0.135, -0.128, -0.463, -0.281, -0.072, -0.479,
       -0.041,  0.155, -0.316, -0.22 , -0.061, -0.188, -0.005,  0.313,
        0.057, -0.162, -0.005,  0.052,  0.04 ,  0.021, -0.062, -0.153,
      

## 9. Apply to new observations for which we have no labels

In [ ]:
Xnew=pd.read_csv('./newobs.csv')

# standardize new Xs in same way
Xnew = scaler.transform(Xnew)
yhatnew=lassowcvalpha.predict(Xnew)
print("predicted residualized ticket sales for new observation: ",yhatnew)

In [ ]:
lassowcvalpha.coef_